# A Notebook for investigating F1 race data

In [1]:
import pymysql
import pymysql.cursors
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

from f1predict.common import file_operations
from f1predict.common import common

In [2]:
USER_VARS = file_operations.getUserVariables("../user_variables.txt")

#Set up a database connection:
connection = pymysql.connect(host='localhost',
                             user=USER_VARS['db_username'],
                             password=USER_VARS['db_password'],
                             db=USER_VARS['db_database'],
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor)

In [3]:
raceResultsByGridPosition = {}

with connection.cursor() as cursor:
    for year in range(2003, common.getCurrentYear() + 1):
        sql = "SELECT `raceId`, `round`, `circuitId`, `name` FROM `races` WHERE `year`=%s"
        cursor.execute(sql, year)
        result = cursor.fetchall()

        for x in result:
            circuitId = x.get('circuitId')        
            sql = "SELECT `driverId`, `constructorId`, `position`, `grid` FROM `results` WHERE `raceId`=%s"
            cursor.execute(sql, x.get('raceId'))
            result = cursor.fetchall()

            if result:
                result.sort(key=lambda result: (result['position'] is None, result['position']))
                for row in result:
                    if row['position']:
                        if row['grid'] not in raceResultsByGridPosition and row['grid'] > 0:
                            raceResultsByGridPosition[row['grid']] = []
                        if row['grid'] > 0:
                            raceResultsByGridPosition[row['grid']].append(row['position'])

In [22]:
with connection.cursor() as cursor:
    sql = "SELECT `status`.status, COUNT(*) AS `count` FROM `status`, `results`, `races` \
        WHERE `status`.statusId = `results`.statusId AND `results`.raceId = `races`.raceId AND `races`.year >= 2003 \
        GROUP BY `results`.statusId \
        ORDER BY COUNT(*) DESC"
    cursor.execute(sql)
    result = cursor.fetchall()
    for x in result:
        if " Lap" not in x.get('status'):
            print("{}: {}".format(x.get('status'), x.get('count')))

Finished: 3245
Collision: 285
Accident: 220
Engine: 174
Gearbox: 101
Hydraulics: 82
Brakes: 74
Spun off: 65
Suspension: 64
Retired: 61
Electrical: 34
Power Unit: 33
Collision damage: 27
Disqualified: 25
Withdrew: 25
Wheel: 24
Transmission: 22
Mechanical: 19
Puncture: 16
Driveshaft: 16
Oil leak: 15
Tyre: 14
Fuel pressure: 14
Clutch: 13
Electronics: 10
Power loss: 10
Overheating: 9
Throttle: 8
Wheel nut: 8
Exhaust: 8
Steering: 7
Fuel system: 6
Water leak: 6
Battery: 5
Out of fuel: 5
ERS: 5
Water pressure: 5
Rear wing: 5
Did not qualify: 4
Vibrations: 4
Technical: 4
Oil pressure: 4
Pneumatics: 4
Turbo: 4
Front wing: 4
Alternator: 3
Radiator: 3
Fuel pump: 2
Track rod: 2
Injured: 2
Heat shield fire: 2
Wheel rim: 2
Excluded: 1
Oil line: 1
Tyre puncture: 1
Fuel rig: 1
Driver Seat: 1
Seat: 1
Launch control: 1
Injury: 1
Not classified: 1
Spark plugs: 1
Broken wing: 1
Fuel: 1
Damage: 1
Differential: 1
Debris: 1
Handling: 1
Drivetrain: 1
Refuelling: 1
Fire: 1
Engine misfire: 1
Engine fire: 1
Brak